## Testing some numba code!!!
I want to incorporate numba into my mind algorithm and don't know how to use it yet...

In [13]:
# inclusions
import numpy as np
import time
import scipy
import numba
from numba import njit, float64, float32, int32, prange, vectorize

In [16]:
def addBasic(x, y):
    return x + y

@njit
def addSimple(x, y):
    # A somewhat trivial example
    return x + y

@njit(parallel=True)
def addParallel(x, y):
    out = np.zeros_like(x)
    for ii in range(len(out)):
        out[ii] = x[ii] + y[ii]
    return out

In [17]:
addBasic(1,2)
addSimple(1,2)
addParallel(1,2)

a = np.random.randint(0,100,(1000,1))
b = np.random.randint(0,100,(1000,1))

%time _ = addBasic(a,b)
%time _ = addSimple(a,b)
%time _ = addParallel(a,b)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(int64, int64)
 
There are 22 candidate implementations:
[1m   - Of which 22 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(int64, int64)':[0m
[1m    No match.[0m
[0m
[0m[1mDuring: typing of intrinsic-call at C:\Users\andrew\AppData\Local\Temp\ipykernel_5060\1175693418.py (13)[0m
[1m
File "..\..\..\AppData\Local\Temp\ipykernel_5060\1175693418.py", line 13:[0m
[1m<source missing, REPL/exec in use?>[0m


NameError: name 'vectorize' is not defined

In [44]:
x = np.arange(100).reshape(10, 10)
numba.typeof(x)

array(int32, 2d, C)

In [65]:
def matMult(A,B):
    return A @ B

N = 1000
A = np.ascontiguousarray(np.random.normal(0,1,(N,N)))
B = np.ascontiguousarray(np.random.normal(0,1,(N,N)))

@jit(float64[:,:](numba.typeof(A),numba.typeof(A)))
def fastMatMult(A,B):
    return A @ B

%timeit matMult(A,B)
%timeit fastMatMult(A,B)

5.82 ms ± 50.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.79 ms ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
@njit((float64, float32))
def ppca(data, minVariance=0.95):
    # probabilistic ppca - using description in Methods Section 1.7 of the following: https://www.biorxiv.org/content/10.1101/418939v2.full.pdf
    # data is a (observations x dimensions) array 
    # default is to set q = to the smallest number of dimensions that exceed some percentage of the variance (default 95%)
    
    # if data.ndim != 2:
    #     raise ValueError("data must be a matrix")
        
    # Return ML estimate of mean
    uML = np.mean(data,axis=0)
    cdata = data - uML # use centered data for computations
    
    # Pick method based on computational speed (logic inherited from Low/Lewallen, haven't tested yet!)
    N,D = data.shape
    if N > D:
        # do eigendecomposition
        covData = np.cov(cdata.T, bias=True)
        w,v = scipy.linalg.eigh(covData)
        w[w<=np.finfo(float).eps]=np.finfo(float).eps # don't allow weird tiny numbers (or negatives, it's a symmetric positive semidefinite matrix)
        idx = np.argsort(-w) # return index of descending sort
        w = w[idx] # sort eigenvalues
        v = v[:,idx] # sort eigenvectors
        s = np.sqrt(N*w) # singular values
        
    else:
        # do svd instead
        _,s,v = np.linalg.svd(cdata)
        v = v.T
        w = s**2 / N # eigenvalues
    
    varExplained = np.cumsum(w / np.sum(w))
    q = int(np.where(varExplained >= minVariance)[0][0])
    
    # Return ML estimate of noise variance
    nvML = np.mean(w[q:]) 
    
    # Keep q eigenvalues & eigenvectors
    w = w[:q]
    v = v[:,:q]
    
    # Compute ML estimate of covariance
    covML = nvML*np.identity(D) + (v @ (np.diag(w) - nvML*np.identity(q)) @ v.T)
    invCovML = np.linalg.inv(covML)
    
    # Return likelihood
    likelihood = -N*D/2*np.log(2*np.pi) - N/2*np.log(np.linalg.det(covML)) - (1/2)*np.sum(np.array([cdata[n,:] @ invCovML @ cdata[n,:].T for n in range(N)]))
    
    return likelihood,uML,covML,nvML,w,v